In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:

dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'


df = pd.read_csv(dataset_path)

#print(df.head())


In [ ]:
import torch
import os
from torchvision import models, transforms
import requests
from PIL import Image, UnidentifiedImageError
import requests
from io import BytesIO
import pandas as pd
import pickle
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import string
from collections import defaultdict, Counter
from math import log
import ast


nltk.download('stopwords')
nltk.download('wordnet')

output_dir = '/content/drive/My Drive/IR_Assignment2'


image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


resnet = models.resnet50(pretrained=True)
resnet.eval()

def extract_image_features(urls):
    features_list = []
    try:

        urls = ast.literal_eval(urls) if isinstance(urls, str) and urls.startswith("[") else [urls]
        for url in urls:
            try:
                response = requests.get(url)
                img = Image.open(BytesIO(response.content))
                img_t = image_transforms(img)
                img_t = img_t.unsqueeze(0)
                with torch.no_grad():
                    features = resnet(img_t)
                features_list.append(features.cpu().numpy().flatten())
            except requests.exceptions.RequestException as e:
                print(f"RequestException for URL {url}: {e}")
            except UnidentifiedImageError:
                print(f"UnidentifiedImageError: cannot identify image file from URL {url}.")
            except Exception as e:
                print(f"Unexpected error for URL {url}: {e}")
    except Exception as e:
        print(f"Error parsing URL string {urls}: {e}")
    return features_list



image_features = []
for index, row in df.iterrows():

    features_list = extract_image_features(row['Image'])
    image_features.extend(features_list)

dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'


df = pd.read_csv(dataset_path)
# Save the results
with open(os.path.join(output_dir, 'image_features_2.pkl'), 'wb') as f:
    pickle.dump(image_features, f)

print(f"Extracted and saved features for images.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/61OboZT-kcL._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/710a2Pyh5lL._SY88.jpg.
UnidentifiedImageError: cannot identify image file from URL https://images-n

In [ ]:
import os
import re
import math
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


nltk.download(['punkt', 'stopwords', 'wordnet'])

def text_normalization_and_processing(input_text):
    """Normalize and process text data for analysis."""

    normalized_text = input_text.lower()

    cleaned_text = re.sub(r'\b(http\S+|@\w+)\b', '', normalized_text)

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(cleaned_text)

    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in english_stopwords]
    return filtered_tokens


english_stopwords = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')


dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'
df = pd.read_csv(dataset_path)

reviews = df['Review Text'].fillna(' ').apply(str).tolist()
processed_reviews = [text_normalization_and_processing(review) for review in reviews]

def tf_idf_computation(docs_tokens):
    """Compute the TF-IDF for each document."""

    tf = [Counter(doc_tokens) for doc_tokens in docs_tokens]

    doc_count = float(len(docs_tokens))
    idf = {}
    for doc in tf:
        for term in doc.keys():
            idf[term] = idf.get(term, 0) + 1
    for term, val in idf.items():
        idf[term] = math.log(doc_count / val)


    tf_idf = []
    for doc in tf:
        doc_tf_idf = {}
        for term, val in doc.items():
            doc_tf_idf[term] = val * idf[term]
        tf_idf.append(doc_tf_idf)

    return tf_idf

tf_idf_scores_refined = tf_idf_computation(processed_reviews)


data_dir = '/content/drive/My Drive/IR_Assignment2'
os.makedirs(data_dir, exist_ok=True)


pickle.dump(processed_reviews, open(os.path.join(data_dir, 'processed_texts.pkl'), 'wb'))
pickle.dump(tf_idf_scores_refined, open(os.path.join(data_dir, 'tf_idf_scores.pkl'), 'wb'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
import numpy as np
import pandas as pd
import pickle
import requests
from PIL import Image
from torchvision import models, transforms
from io import BytesIO
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

# Assuming nltk resources are already downloaded in your environment
# nltk.download(['punkt', 'stopwords', 'wordnet'])

dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'
output_dir = '/content/drive/My Drive/IR_Assignment2'
image_features_path = os.path.join(output_dir, 'image_features.pkl')
tf_idf_scores_path = os.path.join(output_dir, 'tf_idf.pkl')

# Load the dataset and the precomputed features
df = pd.read_csv(dataset_path)

with open(image_features_path, 'rb') as f:
    image_features = pickle.load(f)
with open(tf_idf_scores_path, 'rb') as f:
    tf_idf_scores = pickle.load(f)

resnet_model = models.resnet50(pretrained=True)
resnet_model.eval()

transform_pipeline = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image_url):
    """Fetch and preprocess an image from a URL, then extract features."""
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        processed_image = transform_pipeline(image).unsqueeze(0)
        with torch.no_grad():
            features = resnet_model(processed_image)
        features_np = features.numpy().flatten()
        return features_np
    except Exception as e:
        print(f"Error processing image from URL {image_url}: {e}")
        return None

# Modified cosine similarity function is assumed to be defined here
def cosine_similarity(v1, v2):

  if isinstance(v1, np.ndarray) and all(isinstance(v, np.ndarray) for v in v2):

          v2 = np.array(v2)

          v1_norm = v1 / np.linalg.norm(v1)
          v2_norm = v2 / np.linalg.norm(v2, axis=1)[:, np.newaxis]

          similarities = np.dot(v1_norm, v2_norm.T)


  elif isinstance(v1, dict) and all(isinstance(v, dict) for v in v2):
          similarities = []
          for tfidf_dict in v2:

              common_terms = set(v1.keys()) & set(tfidf_dict.keys())
              dot_product = sum(v1[term] * tfidf_dict[term] for term in common_terms)

              norm_v1 = np.sqrt(sum(value ** 2 for value in v1.values()))
              norm_v2 = np.sqrt(sum(value ** 2 for value in tfidf_dict.values()))
              if norm_v1 == 0 or norm_v2 == 0:
                  similarity = 0
              else:
                  similarity = dot_product / (norm_v1 * norm_v2)
              similarities.append(similarity)
          similarities = np.array(similarities)

  else:
          raise ValueError("Unsupported input types.")

  return similarities

def access_elements_safely(indices, dataset, similarities, description="Item"):
    """Safely access elements by indices, ensuring they are within bounds."""
    for i, idx in enumerate(indices):
        if idx < len(dataset):
            print(f"{description} {i+1}: Index {idx} with similarity {similarities[i]:.4f}")
        else:
            print(f"{description} {i+1}: Index {idx} is out of bounds, skipping.")


input_image_url = input("Enter the image URL: ")
input_review_text = input("Enter the review text: ")


processed_image = preprocess_image(input_image_url)
processed_tokens = preprocess_text(input_review_text)
input_review_tfidf = compute_tf(processed_tokens)


if processed_image is not None:
    similar_image_indices, image_similarities = find_most_similar_images(processed_image, image_features)
    similar_review_indices, review_similarities = find_most_similar_reviews(input_review_tfidf, tf_idf_scores)


    print("Similar Images:")
    access_elements_safely(similar_image_indices, image_features, image_similarities, description="Image")

    print("\nSimilar Reviews:")
    access_elements_safely(similar_review_indices, tf_idf_scores, review_similarities, description="Review")
else:
    print("The specified image URL and review were not found in the dataset, or image processing failed.")


retrieval_results = {
    'similar_image_indices': similar_image_indices,
    'image_similarities': image_similarities,
    'similar_review_indices': similar_review_indices,
    'review_similarities': review_similarities,
}
results_path = os.path.join(output_dir, 'retrieval_results_new.pkl')
with open(results_path, 'wb') as f:
    pickle.dump(retrieval_results, f)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
Enter the review text: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
Similar Images:
Image 1: Index 1238 with similarity 0.9386
Image 2: Index 1094 with similarity 0.8766
Image 3: Index 1621 with similarity 0.8666

Similar Reviews:
Review 1: Index 758 with similarity 0.2609
Review 2: Index 683 with similarity 0.0697
Review 3: Index 622 with similarity 0.0649


In [22]:
import numpy as np
import pickle
from scipy.spatial.distance import cdist
import os
import pandas as pd
output_dir = '/content/drive/My Drive/IR_Assignment2'
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'
df = pd.read_csv(dataset_path)


results_path = os.path.join(output_dir, 'retrieval_results_new.pkl')

with open(results_path, 'rb') as f:
    retrieval_results = pickle.load(f)

similar_image_indices = retrieval_results['similar_image_indices']
image_similarities = retrieval_results['image_similarities']
similar_review_indices = retrieval_results['similar_review_indices']
review_similarities = retrieval_results['review_similarities']


def calculate_composite_scores(image_similarities, review_similarities):
    composite_scores = []
    for image_similarity, review_similarity in zip(image_similarities, review_similarities):

        composite_score = (image_similarity + review_similarity) / 2
        composite_scores.append(composite_score)
    return composite_scores

composite_scores = calculate_composite_scores(image_similarities, review_similarities)

ranked_pairs = sorted(zip(composite_scores, similar_image_indices, similar_review_indices), reverse=True, key=lambda x: x[0])


print("Ranked Combined Retrieval Results:")
for rank, (comp_score, img_idx, rev_idx) in enumerate(ranked_pairs, start=1):
    print(f"Rank: {rank}, Image Index: {img_idx}, Review Index: {rev_idx}, Composite Score: {comp_score:.4f}")


ranked_results_path = os.path.join(output_dir, 'ranked_combined_retrieval_results.pkl')
with open(ranked_results_path, 'wb') as f:
    pickle.dump(ranked_pairs, f)

print(f"Ranked combined retrieval results saved to: {ranked_results_path} \n")

def get_data_by_indices(df, image_indices, review_indices):
    max_index = df.index.max()
    valid_image_indices = [idx for idx in image_indices if idx <= max_index]
    valid_review_indices = [idx for idx in review_indices if idx <= max_index]

    if not valid_image_indices or not valid_review_indices:
        print("Error: Provided indices are out of DataFrame's range.")
        return [], []


    image_urls = df.loc[valid_image_indices, 'Image'].tolist()
    reviews = df.loc[valid_review_indices, 'Review Text'].tolist()
    return image_urls, reviews



Ranked Combined Retrieval Results:
Rank: 1, Image Index: 1238, Review Index: 758, Composite Score: 0.5997
Rank: 2, Image Index: 1094, Review Index: 683, Composite Score: 0.4732
Rank: 3, Image Index: 1621, Review Index: 622, Composite Score: 0.4658
Ranked combined retrieval results saved to: /content/drive/My Drive/IR_Assignment2/ranked_combined_retrieval_results.pkl 



In [27]:
import numpy as np
import pandas as pd
import pickle
import os

output_dir = '/content/drive/My Drive/IR_Assignment2'
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'

df = pd.read_csv(dataset_path)


results_path = os.path.join(output_dir, 'retrieval_results_new.pkl')
with open(results_path, 'rb') as f:
    retrieval_results = pickle.load(f)


similar_image_indices = retrieval_results['similar_image_indices']
image_similarities = retrieval_results['image_similarities']
similar_review_indices = retrieval_results['similar_review_indices']
review_similarities = retrieval_results['review_similarities']


def calculate_composite_scores(image_similarities, review_similarities):
    composite_scores = [(i, (image_similarities[i] + review_similarities[i]) / 2) for i in range(len(image_similarities))]
    composite_scores.sort(key=lambda x: x[1], reverse=True)
    return composite_scores

composite_scores = calculate_composite_scores(image_similarities, review_similarities)


def get_data_by_indices(df, indices):

    valid_indices = [index for index in indices if index < len(df)]
    if not valid_indices:
        return [], []

    image_urls = df.iloc[valid_indices]['Image'].tolist()
    reviews = df.iloc[valid_indices]['Review Text'].tolist()
    return image_urls, reviews


def display_results(df, composite_scores, preprocessed_reviews=None):
    print("USING IMAGE RETRIEVAL")
    for rank, (idx, comp_score) in enumerate(composite_scores, start=1):

        if idx < len(df):
            image_url = df.iloc[idx]['Image'] if 'Image' in df.columns else "Image URL not available"
            review = df.iloc[idx]['Review Text'] if 'Review Text' in df.columns else "Review not available"
            print(f"{rank}) Image URL: {image_url}")
            print(f"Review: {review}")
            print(f"Cosine similarity of images - {image_similarities[idx]:.4f}")
            print(f"Cosine similarity of text - {review_similarities[idx]:.4f}\n")
        else:
            print(f"Skipping rank {rank} due to out-of-bound index.")

    if preprocessed_reviews:
        print("USING TEXT RETRIEVAL")
        for rank, (idx, comp_score) in enumerate(composite_scores, start=1):
            if idx < len(df) and idx < len(preprocessed_reviews):
                image_url = df.iloc[idx]['Image'] if idx < len(df) and 'Image' in df.columns else "Image URL not available"
                preprocessed_review = preprocessed_reviews[idx] if idx < len(preprocessed_reviews) else "Preprocessed review not available"
                print(f"{rank}) Image URL: {image_url}")
                print(f"Preprocessed Review: {preprocessed_review}")
                print(f"Cosine similarity of images - {image_similarities[idx]:.4f}")
                print(f"Cosine similarity of text - {review_similarities[idx]:.4f}\n")
            else:
                print(f"Skipping rank {rank} due to out-of-bound index.")

# Calculate composite scores
composite_scores = calculate_composite_scores(image_similarities, review_similarities)

# Display the results using the modified function
display_results(df, composite_scores, preprocessed_reviews)

# Save your processed reviews and retrieval results as before

final_composite_image_score = sum(image_similarities) / len(image_similarities)
final_composite_text_score = sum(review_similarities) / len(review_similarities)
final_composite_score = (final_composite_image_score + final_composite_text_score) / 2

print("Composite similarity scores of images:", final_composite_image_score)
print("Composite similarity scores of text:", final_composite_text_score)
print("Final composite similarity score:", final_composite_score)

USING IMAGE RETRIEVAL
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']
Review: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Cosine similarity of images - 0.9386
Cosine similarity of text - 0.2609

2) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71HSx4Y-5dL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71dVsYejzTL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71domStNfIL._SY88.jpg']
Review: Works great as a guitar bench mat. Not rugged enough for abuse but if you take care of it, it will take care of you. Makes organization of workspace much easier because screws won't roll around. Color is good too.
Cosine similarity of images - 0.8766
Cosine similarity of text - 0.0697

3) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71Md5ihUFLL._SY